In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

data = {}
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data.update(requests.get(batch_api_call_url).json())



In [4]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

iex_cols = {
    'Price': ['quote','latestPrice'],
    'One-Year Price Return': ['stats', 'year1ChangePercent']
}

final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe['Ticker'] = stocks['Ticker']

for key in iex_cols.keys():
    tmp_series = []
    for stock in stocks['Ticker']:
        tmp_series.append(data[stock][iex_cols[key][0]][iex_cols[key][1]])
    final_dataframe[key] = tmp_series

In [70]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,135.40,0.130866,NaN
1,AAL,13.95,-0.375024,NaN
2,AAP,206.83,0.200092,NaN
3,AAPL,162.64,0.324797,NaN
4,ABBV,148.65,0.481990,NaN
...,...,...,...,...
498,YUM,117.45,0.104598,NaN
499,ZBH,116.46,-0.270587,NaN
500,ZBRA,391.51,-0.207338,NaN
501,ZION,63.48,0.145053,NaN


In [5]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,80.670,2.325653,NaN
1,DVN,61.240,1.670843,NaN
2,OXY,58.900,1.031250,NaN
3,NUE,133.370,1.031063,NaN
4,MRO,24.400,1.010718,NaN
5,MCHP,71.230,0.995959,NaN
6,CF,92.600,0.979370,NaN
7,MOS,58.210,0.908841,NaN
8,NVDA,234.130,0.879004,NaN
9,APA,39.200,0.827553,NaN


In [6]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()

In [7]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,80.670,2.325653,2430
1,DVN,61.240,1.670843,3201
2,OXY,58.900,1.031250,3329
3,NUE,133.370,1.031063,1470
4,MRO,24.400,1.010718,8036
5,MCHP,71.230,0.995959,2752
6,CF,92.600,0.979370,2117
7,MOS,58.210,0.908841,3368
8,NVDA,234.130,0.879004,837
9,APA,39.200,0.827553,5002


In [8]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_iex_cols = {
                'Price': ['quote','latestPrice'],       
                'One-Year Price Return': ['stats', 'year1ChangePercent'],
                'Six-Month Price Return': ['stats', 'month6ChangePercent'],
                'Three-Month Price Return': ['stats', 'month3ChangePercent'],
                'One-Month Price Return': ['stats', 'month1ChangePercent'] 
}

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
hqm_dataframe['Ticker'] = stocks['Ticker']

for key in hqm_iex_cols.keys():
    tmp_series = []
    for stock in stocks['Ticker']:
        tmp_series.append(data[stock][hqm_iex_cols[key][0]][hqm_iex_cols[key][1]])
    hqm_dataframe[key] = tmp_series




In [9]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for time_period in time_periods:
    tmp_series = []
    for row in hqm_dataframe.index:
        tmp_percentile = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
        tmp_series.append(tmp_percentile)
    hqm_dataframe[f'{time_period} Return Percentile'] = tmp_series

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.28,NaN,0.122591,0.520875,-0.250541,0.085487,-0.140958,0.228628,-0.077556,0.399602,NaN
1,AAL,14.70,NaN,-0.345685,0.027833,-0.297910,0.047714,-0.220499,0.101392,-0.246210,0.017893,NaN
2,AAP,206.39,NaN,0.203130,0.667992,0.028720,0.616302,-0.124864,0.292247,-0.121654,0.208748,NaN
3,AAPL,168.09,NaN,0.369009,0.848907,0.063169,0.687873,-0.066313,0.449304,-0.078865,0.393638,NaN
4,ABBV,153.74,NaN,0.472724,0.920477,0.419434,0.952286,0.211992,0.930417,0.040733,0.878728,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,119.43,NaN,0.149496,0.574553,-0.095935,0.332008,-0.128312,0.268390,-0.088883,0.351889,NaN
499,ZBH,118.49,NaN,-0.256251,0.067594,-0.192438,0.149105,-0.063723,0.461233,-0.021030,0.679920,NaN
500,ZBRA,403.20,NaN,-0.152934,0.143141,-0.320759,0.033797,-0.333746,0.015905,-0.213087,0.029821,NaN
501,ZION,66.89,NaN,0.186794,0.642147,0.150979,0.821074,0.039363,0.715706,-0.106870,0.258449,NaN


In [10]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.28,NaN,0.122591,0.520875,-0.250541,0.085487,-0.140958,0.228628,-0.077556,0.399602,0.308648
1,AAL,14.70,NaN,-0.345685,0.027833,-0.297910,0.047714,-0.220499,0.101392,-0.246210,0.017893,0.048708
2,AAP,206.39,NaN,0.203130,0.667992,0.028720,0.616302,-0.124864,0.292247,-0.121654,0.208748,0.446322
3,AAPL,168.09,NaN,0.369009,0.848907,0.063169,0.687873,-0.066313,0.449304,-0.078865,0.393638,0.59493
4,ABBV,153.74,NaN,0.472724,0.920477,0.419434,0.952286,0.211992,0.930417,0.040733,0.878728,0.920477
5,ABC,151.11,NaN,0.356767,0.840954,0.197283,0.876740,0.210758,0.928429,0.024637,0.852883,0.874751
6,ABMD,315.67,NaN,0.033943,0.363817,-0.169087,0.190855,-0.017779,0.558648,-0.030268,0.620278,0.4334
7,ABT,118.60,NaN,0.030864,0.355865,-0.071289,0.409543,-0.105735,0.343936,-0.096122,0.306163,0.353877
8,ACN,311.11,NaN,0.236818,0.721670,-0.089528,0.343936,-0.169361,0.184891,-0.132760,0.172962,0.355865
9,ADBE,461.44,NaN,0.027534,0.347913,-0.322528,0.029821,-0.293098,0.033797,-0.138384,0.153082,0.141153


In [11]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.28,1417,0.122591,0.520875,-0.250541,0.085487,-0.140958,0.228628,-0.077556,0.399602,0.308648
1,AAL,14.70,13338,-0.345685,0.027833,-0.297910,0.047714,-0.220499,0.101392,-0.246210,0.017893,0.048708
2,AAP,206.39,950,0.203130,0.667992,0.028720,0.616302,-0.124864,0.292247,-0.121654,0.208748,0.446322
3,AAPL,168.09,1166,0.369009,0.848907,0.063169,0.687873,-0.066313,0.449304,-0.078865,0.393638,0.59493
4,ABBV,153.74,1275,0.472724,0.920477,0.419434,0.952286,0.211992,0.930417,0.040733,0.878728,0.920477
5,ABC,151.11,1297,0.356767,0.840954,0.197283,0.876740,0.210758,0.928429,0.024637,0.852883,0.874751
6,ABMD,315.67,621,0.033943,0.363817,-0.169087,0.190855,-0.017779,0.558648,-0.030268,0.620278,0.4334
7,ABT,118.60,1653,0.030864,0.355865,-0.071289,0.409543,-0.105735,0.343936,-0.096122,0.306163,0.353877
8,ACN,311.11,630,0.236818,0.721670,-0.089528,0.343936,-0.169361,0.184891,-0.132760,0.172962,0.355865
9,ADBE,461.44,424,0.027534,0.347913,-0.322528,0.029821,-0.293098,0.033797,-0.138384,0.153082,0.141153


In [14]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)


In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

    writer.save()